## iReady Data






In [ ]:
storage_account = 'steduanalytics__update_this'
use_test_env = True

In [ ]:
if use_test_env:
    stage1 = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage1'
    stage2 = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage2'
    stage3 = 'abfss://test-env@' + storage_account + '.dfs.core.windows.net/stage3'
else:
    stage1 = 'abfss://stage1@' + storage_account + '.dfs.core.windows.net'
    stage2 = 'abfss://stage2@' + storage_account + '.dfs.core.windows.net'
    stage3 = 'abfss://stage3@' + storage_account + '.dfs.core.windows.net'

In [ ]:
# Process personalized_instruction_by_lesson_math.csv
def remove_spaces(str): return str.replace(' ', '').replace('(','_').replace(')','_').replace('=', '__')

def process(filename):
  df = spark.read.csv(stage1 + '/iready/' + filename + '.csv', header='true', inferSchema='true')
  newColumns = map(remove_spaces, df.columns)
  df = df.toDF(*newColumns)
  df = df.withColumn('StudentID',df.StudentID.cast('string')) # StudentID needs to be a string to allow for hashing when moving into stage3
  df.write.format('parquet').mode('overwrite').save(stage2 + '/iready/' + filename)

process('comprehensive_student_lesson_activity_with_standards_ela')
process('comprehensive_student_lesson_activity_with_standards_math')
process('diagnostic_and_instruction_ela_ytd_window')
process('diagnostic_and_instruction_math_ytd_window')
process('diagnostic_results_ela')
process('diagnostic_results_math')
process('personalized_instruction_by_lesson_ela')
process('personalized_instruction_by_lesson_math')

# Anonymize data and load into stage3
from pyspark.sql.functions import sha2, lit

df = spark.read.format('parquet').load(stage2 + '/iready/comprehensive_student_lesson_activity_with_standards_ela')
df = df.withColumn('StudentID', sha2(df.StudentID, 256)).withColumn('LastName',lit('*')).withColumn('FirstName',lit('*'))
df.write.format('parquet').mode('overwrite').save(stage3 + '/iready/comprehensive_student_lesson_activity_with_standards_ela')

df = spark.read.format('parquet').load(stage2 + '/iready/comprehensive_student_lesson_activity_with_standards_math')
df = df.withColumn('StudentID', sha2(df.StudentID, 256)).withColumn('LastName',lit('*')).withColumn('FirstName',lit('*'))
df.write.format('parquet').mode('overwrite').save(stage3 + '/iready/comprehensive_student_lesson_activity_with_standards_math')

df = spark.read.format('parquet').load(stage2 + '/iready/diagnostic_and_instruction_ela_ytd_window')
df = df.withColumn('StudentID', sha2(df.StudentID, 256)).withColumn('LastName',lit('*')).withColumn('FirstName',lit('*')).withColumn('UserName', lit('*'))
df.write.format('parquet').mode('overwrite').save(stage3 + '/iready/diagnostic_and_instruction_ela_ytd_window')

df = spark.read.format('parquet').load(stage2 + '/iready/diagnostic_and_instruction_math_ytd_window')
df = df.withColumn('StudentID', sha2(df.StudentID, 256)).withColumn('LastName',lit('*')).withColumn('FirstName',lit('*')).withColumn('UserName', lit('*'))
df.write.format('parquet').mode('overwrite').save(stage3 + '/iready/diagnostic_and_instruction_math_ytd_window')

df = spark.read.format('parquet').load(stage2 + '/iready/diagnostic_results_ela')
df = df.withColumn('StudentID', sha2(df.StudentID, 256)).withColumn('LastName',lit('*')).withColumn('FirstName',lit('*'))
df.write.format('parquet').mode('overwrite').save(stage3 + '/iready/diagnostic_results_ela')

df = spark.read.format('parquet').load(stage2 + '/iready/diagnostic_results_math')
df = df.withColumn('StudentID', sha2(df.StudentID, 256)).withColumn('LastName',lit('*')).withColumn('FirstName',lit('*'))
df.write.format('parquet').mode('overwrite').save(stage3 + '/iready/diagnostic_results_math')

df = spark.read.format('parquet').load(stage2 + '/iready/personalized_instruction_by_lesson_ela')
df = df.withColumn('StudentID', sha2(df.StudentID, 256)).withColumn('LastName',lit('*')).withColumn('FirstName',lit('*'))
df.write.format('parquet').mode('overwrite').save(stage3 + '/iready/personalized_instruction_by_lesson_ela')

df = spark.read.format('parquet').load(stage2 + '/iready/personalized_instruction_by_lesson_math')
df = df.withColumn('StudentID', sha2(df.StudentID, 256)).withColumn('LastName',lit('*')).withColumn('FirstName',lit('*'))
df.write.format('parquet').mode('overwrite').save(stage3 + '/iready/personalized_instruction_by_lesson_math')

In [ ]:
# Create sql on-demand db's for iReady data
def create_spark_db(db_name, source_path):
    spark.sql('CREATE DATABASE IF NOT EXISTS ' + db_name)
    spark.sql("create table if not exists " + db_name + ".comprehensive_student_lesson_activity_with_standards_ela using PARQUET location '" + source_path + "/comprehensive_student_lesson_activity_with_standards_ela'")
    spark.sql("create table if not exists " + db_name + ".comprehensive_student_lesson_activity_with_standards_math using PARQUET location '" + source_path + "/comprehensive_student_lesson_activity_with_standards_math'")
    spark.sql("create table if not exists " + db_name + ".diagnostic_and_instruction_ela_ytd_window using PARQUET location '" + source_path + "/diagnostic_and_instruction_ela_ytd_window'")
    spark.sql("create table if not exists " + db_name + ".diagnostic_and_instruction_math_ytd_window using PARQUET location '" + source_path + "/diagnostic_and_instruction_math_ytd_window'")
    spark.sql("create table if not exists " + db_name + ".diagnostic_results_ela using PARQUET location '" + source_path + "/diagnostic_results_ela'")
    spark.sql("create table if not exists " + db_name + ".diagnostic_results_math using PARQUET location '" + source_path + "/diagnostic_results_math'")
    spark.sql("create table if not exists " + db_name + ".personalized_instruction_by_lesson_ela using PARQUET location '" + source_path + "/personalized_instruction_by_lesson_ela'")
    spark.sql("create table if not exists " + db_name + ".personalized_instruction_by_lesson_math using PARQUET location '" + source_path + "/personalized_instruction_by_lesson_math'")

db_prefix = 'test_' if use_test_env else ''
create_spark_db(db_prefix + 's2_iready', stage2 + '/iready')
create_spark_db(db_prefix + 's3_iready', stage3 + '/iready')